In [3]:
##############################################################################################################"Implémentation de la classe point"  
class Point:  #on crée une classe point à laquelle on donne une abcisse et une ordonnée
    def __init__(self, x, y, car=None): #initialisation
        self.x, self.y = x, y
        self.car = car
                             
    def __repr__(self): #retourne les caracterisqtiques du point
        return '{}: {}'.format(str((self.x, self.y)), repr(self.car))
    
    def __str__(self): #retourne les caracteristiques formatés du point
        return 'P({:.2f}, {:.2f})'.format(self.x, self.y)
                             
    def distance_to(self, other): ##calcule la distance entre deux points
        try:
            other_x, other_y = other.x, other.y
        except AttributeError:
            other_x, other_y = other
        return np.hypot(self.x - other_x, self.y - other_y)
                             
##########################################################################Definition de la classe rectangle
class Rect: #rectangle centré en (cx,cy) #####(w,h) symbolise la longueur et la largeur du rectangleon lui donne deux attributs qui donnent les coordonnées de ses points limites

    def __init__(self, cx, cy, w, h):
        self.cx, self.cy = cx, cy
        self.w, self.h = w, h
        self.coin_gauche, self.coin_droit = cx - w/2, cx + w/2
        self.coin_nord, self.coin_sud = cy - h/2, cy + h/2
                             
    def __repr__(self):#Donne les valeurs limites du rectangle
        return str((self.coin_gauche, self.coin_droit, self.coin_nord,
                self.coin_sud))
                             
    def contiens(self, point): #teste l'appartenance d'un point a un rectangle donné
        try:
            point_x, point_y = point.x, point.y
        except AttributeError:
            point_x, point_y = point
        return (point_x >= self.coin_gauche and
                point_x <  self.coin_droit and
                point_y >= self.coin_cord and
                point_y < self.coin_sud)
                             
    def intersecte(self, other): #On verifie si ce rectangle a une intersection avec un autre rectangle
        return not (other.coin_gauche > self.coiin_droit or
                    other.coin_droit < self.coin_gauche or
                    other.coin_nord > self.coin_sud or
                    other.coin_sud < self.coin_nord)
                             
    def dessinne(self, ax, c='k', lw=1, **kwargs):#On déssine le carré avec les dimentions definies comme on l'a fait en haut
        x1, y1 = self.coin_gauche, self.coin_nord
        x2, y2 = self.coin_droit, self.coin_sud
        ax.plot([x1,x2,x2,x1,x1],[y1,y1,y2,y2,y1], c=c, lw=lw, **kwargs)


                             
#######################################################"Implémentation de la classe quadtree 
class QuadTree:
  
    def __init__(self, limites, max_points=4, profondeur=0): #on initialise le noeud du quadtree limites est un objet de type rect qui représente une region du rectangle
                             #ou l'on placera les points à l'interieur de la region dans un noeud, max_point est le nombre de points maximum par noeud qui est de 4, profondeur nous donne la profondeur du noeud 
#divisé nous dis si le noeud s'est divisé ou non

        self.limite = limite
        self.max_points = max_points
        self.points = []
        self.profondeur = profondeur
        self.divisé = False

                             
                             
    def __str__(self):#nous retourne un string detaillant les caracteristiqes du noeud
        sp = ' ' * self.profondeur * 2
        s = str(self.limite) + '\n'
        s += sp + ', '.join(str(point) for point in self.points)
        if not self.divisé:
            return s
        return s + '\n' + '\n'.join([
                sp + 'nw: ' + str(self.nw), sp + 'ne: ' + str(self.ne),
                sp + 'se: ' + str(self.se), sp + 'sw: ' + str(self.sw)])
    
                             
    def divise(self): #les limites des enfants sont dans les 4 quadrants nord est nord ouest , sud ouest ,sud est on divise de la bonne manierre

        cx, cy = self.limite.cx, self.limite.cy
        w, h = self.limite.w / 2, self.limite.h / 2
        self.nw = QuadTree(Rect(cx - w/2, cy - h/2, w, h),
                                    self.max_points, self.depth + 1)
        self.ne = QuadTree(Rect(cx + w/2, cy - h/2, w, h),
                                    self.max_points, self.depth + 1)
        self.se = QuadTree(Rect(cx + w/2, cy + h/2, w, h),
                                    self.max_points, self.depth + 1)
        self.sw = QuadTree(Rect(cx - w/2, cy + h/2, w, h),
                                    self.max_points, self.depth + 1)
        self.divisé = True
                             
    def insert(self, point):
    #insérer un point dans le quadtree

        if not self.limite.contains(point):
            #si le point n'est pas dans la limite on retourne false
            return False
        if len(self.points) < self.max_points:
            # On peut rajouter un point dans le noeud
            self.points.append(point)
            return True

        # si pas possible on divise a nouveau le noeud puis on teste l'insertion dans le noeud enfant
        if not self.divisé:
            self.divise()

        return (self.ne.insert(point) or
                self.nw.insert(point) or
                self.se.insert(point) or
                self.sw.insert(point))

                             
                             
    def recherche(self, limite, pointstrouvés):
        #trouver les points du quadtre

        if not self.limite.intersects(limite):
            #Si la limite de la representation du noeud n'a pas d'intersection avec la limite on ne recherche pas de points .
            return False

        # On teste pour voir si l'on retrouve les points du noeud dans la limite
        for point in self.points:
            if limite.contains(point):
                pointstrouvés.append(point)
        # si le noeud a des enfants on les cherches aussi
        if self.divisé:
            self.nw.recherche(limite, pointstrouvés)
            self.ne.recherche(limite, pointstrouvés)
            self.se.recherche(limite, pointstrouvés)
            self.sw.recherche(limite, pointstrouvés)
        return pointstrouvés
                             
        
                             
        
    def __len__(self):
    #On recherche le nombre de points dans le quadtree

        npoints = len(self.points)
        if self.divisé:
            npoints += len(self.nw)+len(self.ne)+len(self.se)+len(self.sw)
        return npoints
                             
                             
    
    def dessine(self, ax):
#déssine une représentation du quadtree

        self.limite.dessine(ax)
        if self.divisé:
            self.nw.dessine(ax)
            self.ne.dessine(ax)
            self.se.dessine(ax)
            self.sw.dessine(ax)

    

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

DPI = 72
np.random.seed(60)

width, height = 600, 400

N = 500
coords = np.random.randn(N, 2) * height/3 + (width/2, height/2)
points = [Point(*coord) for coord in coords]

domain = Rect(width/2, height/2, width, height)
qtree = QuadTree(domain, 3)
for point in points:
    qtree.insert(point)

print('Number of points in the domain =', len(qtree))

fig = plt.figure(figsize=(700/DPI, 500/DPI), dpi=DPI)
ax = plt.subplot()
ax.set_xlim(0, width)
ax.set_ylim(0, height)
qtree.dessine(ax)

ax.scatter([p.x for p in points], [p.y for p in points], s=4)
ax.set_xticks([])
ax.set_yticks([])

region = Rect(140, 190, 150, 150)
found_points = []
qtree.recherche(region, found_points)
print('Number of found points =', len(pointstrouvés))

ax.scatter([p.x for p in found_points], [p.y for p in pointstrouvés],
           facecolors='none', edgecolors='r', s=32)

region.draw(ax, c='r')

ax.invert_yaxis()
plt.tight_layout()
plt.savefig('search-quadtree.png', DPI=72)
plt.show()

NameError: name 'limite' is not defined